<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> My first BVP </h2>
    <h2> <a href="#acknowledgements"> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.01</h2>
</center>

<div id='toc' />

## Table of Contents

* [My first BVP](#myfirstbvp)
* [Acknowledgements](#acknowledgements)

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt # type: ignore
from matplotlib import pyplot # type: ignore
import numpy as np # type: ignore

from scipy.integrate import solve_ivp # type: ignore
from scipy import optimize # type: ignore 
from pylab import * # type: ignore
from numpy import linalg as LA # type: ignore
from matplotlib.legend_handler import HandlerLine2D # type: ignore
from scipy.linalg import toeplitz # type: ignore
from scipy.optimize import root # type: ignore
from ipywidgets import interact, RadioButtons, Checkbox # type: ignore
import sympy as sym # type: ignore
import matplotlib as mpl # type: ignore
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
from mpl_toolkits.axes_grid1 import make_axes_locatable # type: ignore
sym.init_printing()

# This function builds the h-less differentiation matrices for
# the approximation of the first and second derivatives.
# h-less means that it still needs to add the corresponding
# h coefficient in the approximation.
def build_D_D2(M):
    # First derivative - Central difference differentiation matrix
    D  = toeplitz(np.append(np.array([0, -1.]), np.zeros(M-2)), 
                     np.append(np.array([0, 1.]), np.zeros(M-2)))
    # Second derivative - differentiation matrix
    D2 = toeplitz(np.append(np.array([-2, 1.]), np.zeros(M-2)))
    return D, D2

# Forward Euler Method
def eulerMethod_one_step(yi,ti,f,h):
    return yi+h*f(ti,yi)

# Backward Euler for 1D and nD problems
def backwardEuler_one_step(yi,ti,f,h):
    f_hat = lambda x:  x - yi - f(ti+h,x)*h
    if isinstance(yi,(int,float)):
        out = optimize.root_scalar(f_hat,x0=yi)
        if out.converged:
            return out.root
        else:
            raise Exception("Backward Euler 1D: We couldn't find the root. Select/build another solver.")
    else:
        out = optimize.root(f_hat,yi)
        if out.success:
            return out.x
        else:
            raise Exception("Backward Euler nD: We couldn't find the root. Select/build another solver.")

# Runge-Kutta of Second order
def RK2_one_step(yi,ti,f,h):
    k1=f(ti,yi)
    return yi+h*f(ti+h/2.0,yi+h/2.0*k1)

# Runge-Kutta 
def RK4_one_step(yi,ti,f,h):
    k1=f(ti,yi)
    k2=f(ti+h/2.0,yi+(h/2.0)*k1)
    k3=f(ti+h/2.0,yi+(h/2.0)*k2)
    k4=f(ti+h,yi+h*k3)
    return yi+(h/6.0)*(k1+2.0*k2+2.0*k3+k4)

def eulerMethod(t0,T,N,y0,f):
    t = np.linspace(t0,T,N+1)
    h = (T-t0)/N
    if isinstance(y0,(int,float)):
        y = np.zeros(N+1)
    else:
        y = np.zeros((N+1,len(y0)))
    y[0] = y0
    for i in np.arange(N):
        y[i+1] = eulerMethod_one_step(y[i],t[i],f,h)
    return t, y

def backwardEulerMethod(t0,T,N,y0,f):
    t = np.linspace(t0,T,N+1)
    h = (T-t0)/N
    if isinstance(y0,(int,float)):
        y = np.zeros(N+1)
    else:
        y = np.zeros((N+1,len(y0)))
    y[0] = y0
    for i in np.arange(N):
        y[i+1] = backwardEuler_one_step(y[i],t[i],f,h)
    return t, y

def RK2(t0,T,N,y0,f):
    t = np.linspace(t0,T,N+1)
    h = (T-t0)/N
    if isinstance(y0,(int,float)):
        y = np.zeros(N+1)
    else:
        y = np.zeros((N+1,len(y0)))
    y[0] = y0
    for i in np.arange(N):
        y[i+1] = RK2_one_step(y[i],t[i],f,h)
    return t, y

def RK4(t0,T,N,y0,f):
    t = np.linspace(t0,T,N+1)
    h = (T-t0)/N
    if isinstance(y0,(int,float)):
        y = np.zeros(N+1)
    else:
        y = np.zeros((N+1,len(y0)))
    y[0] = y0
    for i in np.arange(N):
        y[i+1] = RK4_one_step(y[i],t[i],f,h)
    return t, y

radio_button_ODEsolvers=RadioButtons(
    options=[('Euler\'s Method',eulerMethod),('Backward Euler Method',backwardEulerMethod),('RK2',RK2),('RK4',RK4)],
    value=eulerMethod,
    description='ODE solver:',
    disabled=False
)

<div id='bvp' />

# Boundary Value Problems (BVP)
[Back to TOC](#toc)

The following is a initial definition of a second order linear BVP:
$$ 
\begin{align*}
	a(x)\,y''(x)+b(x)\,y'(x)+c(x)\,y(x)&=f(x), \,\, \textrm{for } x\in]0,1[,\\
	y(0)&=y_{\textrm{left}},\\
	y(1)&=y_{\textrm{right}},
\end{align*}
$$
where $a(x)$, $b(x)$, $c(x)$, and $f(x)$ are known and continuous functions.
We also know coefficients for the boundary conditions, i.e. $y_{\textrm{left}}$ and $y_{\textrm{right}}$.


<div id='myfirstbvp' />

# My first BVP - The ODE
[Back to TOC](#toc)

$$
\begin{align*}
    y''(x) &= f_{\textrm{RHS}}(x) = x, \quad x\in]0,1[,&& \leftarrow \textrm{The equation/model and its domain},\\
    y(0) &= -1, && \leftarrow \textrm{The \textbf{left} boundary condition},\\
    y(1) &= 2, && \leftarrow \textrm{The \textbf{right} boundary condition}.
\end{align*}
$$
We point out that we used $f_{\textrm{RHS}}(x)=x$ in the previous explanation for two reasons:
1. When $f_{\textrm{RHS}}(x)=x$, we can integrate the right-hand-side very easily (which is what we do below).
2. When $f_{\textrm{RHS}}(x)$ is just a general function, we may not be able to integrate it easily but we can solve it numerically! This mean to solve numerically the BVP!

# My first BVP - Theoretical Analysis
We first begin by solving the ODE algebraically (this is not generally possible, but it is for this simple case!),
$$
\begin{align*}
    y'(x) &= \dfrac{x^2}{2}+c_1, && \textrm{integrating with respect to $x$.}\\
    y(x) &= \dfrac{x^3}{6}+c_1\,x+c_2, && \textrm{integrating again with respect to $x$.}\\ 
    y(0) &= c_2 = -1, && \textrm{applying the left boundary condition implies that $c_2=-1$.}\\
    y(1) &= \dfrac{1}{6}+c_1-1 = 2, && \textrm{applying the right boundary condition implies that $c_1=\dfrac{17}{6}$.}\\
    y(x) &= \dfrac{x^3}{6}+\dfrac{17}{6}\,x-1, && \textrm{the ``algebraic'' solution.}\\
\end{align*}
$$
Thus, we now can compare any **numerical approximations** with the **algebraic solution**.

## The Numerical Methods availables

For solving BVP we have available two methods:
1. The Shooting Method
    - This method uses what we learned about numerical solvers for IVP to numerical solve BVPs.
    - It is very straight forward to use but it requires to determine an _additional_ initial condition.
    - To find the additional condition, we need to find a root of a _numerical_ function.
2. Finite Differences
    - It is a numerical method specially design for BVP.
    - It _translate_ the ODE from its continuous version to a discrete version.
    - The discrete version of the ODE preserves the nature of the ODE, if it is linear, it generates a linear system of equation and if it is non-linear, it generates a nonlinear system of equations.
    - The boundary conditions are usually _explicitly_ defined, as we showed before with $y(0)=y_{\textrm{left}}$ and $y(1)=y_{\textrm{right}}$, however they could have different representations. For instance, $y'(0)=c_l$, in this case we would need to approximate the derivate with a _forward difference_ (since it is the left boundary condition) first, and then include it in to the linear or nonlinear system of equations, respectively.
    - This methods has two advantages over the Shooting Method:
        1. It does not need to find a root, which implies that it does not need to solve and IVP several times.
        2. It can handle larger values of $h$.



## Solving the BVP with the Shooting Method

We **first** step is to translate the BVP into a IVP,
$$
\begin{align*}
    \ddot{y}(t) &= t, \quad t\in]0,1[,\\
    y(0) &= -1,\\
    \dot{y}(0) &= \alpha, && \leftarrow \textrm{the \textbf{missing} initial condition}.
\end{align*}
$$
The **second** step is to build the corresponding Dynamical system with the following change of variables,
$$
\begin{align*}
    y_1(t) &= y(t),\\
    y_2(t) &= \dot{y}(t),
\end{align*}
$$
which implies that $y_1(0)=y(0)=-1$ in this case, and $y_2(0)=\dot{y}(0)$, which is unfortunately **unknown**, and we will denote it as $\alpha$.
Now, we conpute the derivative with respect to $\textrm{t}$ for both equations and we get,
$$
\begin{align*}
    \dot{y}_1(t) &= \dot{y}(t) = y_2(t), && \textrm{notice we replaced $y_2(t)$ right away since, by definition, it is equal to $\dot{y}(t)$.}\\
    \dot{y}_2(t) &= \ddot{y}(t) = t, && \textrm{in this case we replace again right away since we know $\ddot{y}(t) = t$.}
\end{align*}
$$
Thus, fincally, the Dynamical System is,
$$
\begin{align*}
    \dot{y}_1(t) &= y_2(t),\\
    \dot{y}_2(t) &= t,\\
    y_1(0) &= -1,\\
    y_2(0) &= \alpha.
\end{align*}
$$
Or in the vector form,
$$
\begin{align*}
    \dot{\mathbf{y}} &= \mathbf{F}(t,\mathbf{y})= \begin{bmatrix} y_2\\t\end{bmatrix},\\
    \mathbf{y}(0) &= \begin{bmatrix} -1 \\ \alpha \end{bmatrix}.\\
\end{align*}
$$

The **third** step corresponde to determine $\alpha$.
This step requires to select $\alpha$ such that the numerical approximation obtained for $y_1(t)$ at $t=1$ is equal to $2$, since it is the right boundary condition.
When we acomplish this, we can conclude that the numerical solution obtained for $y_1(t)$ is the numerical approximation for $y(x)$!


## My First BVP - Shooting Method - Numerical Computation

In [2]:
functions_for_f_RHS=[(0,lambda x: x),
              (1,lambda x: 10*np.sin(10*x)),
              (2,lambda x: np.exp(3*x)),
              (3,lambda x: 1/(1+x**2)),
              (4,lambda x: 10*np.sqrt(1.+x+np.cos(x)))]
labels = [r'x',
          r'10*sin(10*x)',
          r'exp(3\,x)',
          r'1/(1+x^2)',
          r'10*sqrt{1+x+cos(x)}']
data=zip(labels,functions_for_f_RHS)

radio_button_function=RadioButtons(
    options=list(data),
    description='Function:',
    disabled=False
)

def plot_numerical_solution_shooting_method(f_input, alpha=0,N=10,L=5,ODEsolver=eulerMethod,flag_y2=True):
    
    f_RHS_id, f_RHS = f_input
    
    # Defining RHS of IVP
    F = lambda t,y : np.array([y[1], f_RHS(t)])
    
    y0=np.array([-1, alpha])
    t_times, y_output = ODEsolver(0,1,N,y0,F)
    
    plt.figure(figsize=(L,L))
    
    plt.title(r'Error=$y_{\mathrm{right}}-y1_N$=2-%.2f=%.3f'%(y_output[-1,0],2-y_output[-1,0]))
    plt.plot(t_times,y_output[:,0],'r.',ms=12,alpha=0.5,label=r'$y1_i$')
    if flag_y2:
        plt.plot(t_times,y_output[:,1],'m.',ms=12,alpha=0.5,label=r'$y2_i$')
    
    plt.plot([1,1],[2,y_output[-1,0]],'k-.',
             linewidth=3,label=r'Error on $\mathrm{right}$ BC',
             alpha=0.5)
    
    if f_RHS_id == 0:
        y_exact = lambda t: np.power(t,3)/6.+(17.*t)/6-1.
        y_prime_exact = lambda t: np.power(t,2)/2.+17./6
        tt = np.linspace(0,1,100)
        plt.plot(tt, y_exact(tt),'b-',label=r'$y(t)$')
        if flag_y2:
            plt.plot(tt, y_prime_exact(tt),'b--',label=r'$\dfrac{\mathrm{d}y}{\mathrm{d}t}(t)$')
    
    font = {'horizontalalignment':'center','verticalalignment':'top'}
    plt.text(1-0.02,y_output[-1,0]-0.1,r'$y1_N$',fontdict=font)
    
    plt.plot(0,-1,'*r',ms=16,label='Left BC')
    plt.plot(1,2,'*g',ms=16,label='Right BC')
    plt.plot(0,alpha+0.2,'darkorange',ms=16,label=r'$\alpha$', marker=r'$\alpha$')
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

interact(plot_numerical_solution_shooting_method, f_input=radio_button_function, alpha=(-10,10,0.01), 
         N=(5,100,1), L=(3,10,1), ODEsolver=radio_button_ODEsolvers)

interactive(children=(RadioButtons(description='Function:', options=(('x', (0, <function <lambda> at 0x1366c1e…

<function __main__.plot_numerical_solution_shooting_method(f_input, alpha=0, N=10, L=5, ODEsolver=<function eulerMethod at 0x1366c16c0>, flag_y2=True)>

## Solving the BVP with the Finite Difference Method
In this case we need to remember how to approximate a derivative from a _numerical/discrete_ representation of a function.

In general, one can consider a function as $f(x)$ where $x\in[a,b]$.
Now, a discrete version of $f(x)$ can be seen as tuple $((x_0,f_0),(x_1,f_1),\dots,(x_N,f_N))$, where $f_i\approx f(x_i)$.
This means we only know a _discrete_ set of values of $f(x)$.

Now, in this case we will use $y_i$, which approximate $y(x_i)$.
The next challenge is to obtain a **numerical approximation** of the first and second derivatives:
$$
\begin{align*}
    y'(x_i) &= \dfrac{y(x_i+h)-y(x_i)}{h} + \mathcal{O}(h), &\textbf{ Forward Difference}. \\
    y'(x_i) &= \dfrac{y(x_i)-y(x_i-h)}{h} + \mathcal{O}(h), &\textbf{ Backward Difference}. \\
    y'(x_i) &= \dfrac{y(x_i+h)-y(x_i-h)}{2\,h} +\mathcal{O}(h^2), &\textbf{ Central Difference}.
\end{align*}
$$
And
$$
\begin{align*}
    y''(x_i) &=\dfrac{y(x_i+h)-2\,y(x_i)+y(x_i-h)}{h^2}+\mathcal{O}(h^2).
\end{align*}
$$

The idea now is as follows:
1. Define a discrete version of the function we are approximating: $\{y_0,y_1,\dots,y_N\}$. Note that we did not use definition as a tuple used before because we can store $y_i$ and $x_i$, for $i\in\{0,1,2,\dots,N\}$, separately.
2. Use the boundary conditions to obtain the values for $y_0$ and $y_N$, which corresponds to,
    - $y(0)=-1 = y_0$
    - $y(1)=2 = y_N$
3. Use the ODE to relate the other unknowns terms, i.e. $y_i$. Recall that the ODE is valid **inside** the domain, i.e. $y''(x) = x$ for $x \in ]0,1[$. Note that the extreme values of the domain are not included. So, since it is valid inside the domain we can instance it at any $x_i$, i.e. we get the following equation $y''(x_i)=x_i$. Thus, for each $x_i$ we obtain:
    - $y''(x_1)=x_1$, which induces the following discrete approximation $\dfrac{y_2-2\,y_1+y_0}{h^2}=x_1$.
    - $y''(x_2)=x_2$, $\rightarrow$ $\dfrac{y_3-2\,y_2+y_1}{h^2}=x_2$. 
    - $\vdots$
    - $y''(x_i)=x_i$, $\rightarrow$ $\dfrac{y_{i+1}-2\,y_i+y_{i-1}}{h^2}=x_i$.
    - $\vdots$
    - $y''(x_{N-1})=x_{N-1}$, $\rightarrow$ $\dfrac{y_N-2\,y_{N-1}+y_{N-2}}{h^2}=x_{N-1}$.
4. Re-write the problem as, in this case, linear system of equations:
    - $y_2-2\,y_1=h^2\,x_1-y_0$
    - $y_3-2\,y_2+y_0=h^2\,x_2$
    - $\vdots$
    - $y_{i+1}-2\,y_i+y_{i-1}=h^2\,x_i$
    - $\vdots$
    - $-2\,y_{N-1}+y_{N-2}=h^2\,x_{N-1}-y_N$
    
    or in the matrix form,
    $$
    D_2\,\widetilde{\mathbf{y}}=h^2\,\widetilde{\mathbf{x}} - \begin{bmatrix}y_0\\
    0\\
    \vdots\\
    0\\
    y_N\end{bmatrix}
    $$, 
    
    where $\widetilde{\mathbf{y}}=[y_1,y_2,\dots,y_{N-1}]$,
    $\widetilde{\mathbf{x}}=[x_1,x_2,\dots,x_{N-1}]$, and
    
    $$
    D_2 = 
    \begin{bmatrix}
        -2 &  1 & 0 & 0 & 0 & 0 & 0 \\
         1 & -2 & 1 & 0 & 0 & 0 & 0 \\
         0 &  1 & -2 & 1 & 0 & 0 & 0 \\
         \vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \ddots \\
         0 &  0 & 0 & 0 & 1 & -2 & 1 \\
         0 &  0 & 0 & 0 & 0 & 1 & -2 \\
    \end{bmatrix}.
    $$

## My First BVP - Finite Difference Method - Numerical Computation

In [3]:
def plot_numerical_solution_finite_difference(f_input, N=5,L=5):
    
    f_RHS_id, f_RHS = f_input
    
    _, D2 = build_D_D2(N-1)
    x = np.linspace(0,1,N+1)
    y = np.zeros(N+1)
    y[0] = -1
    y[N] = 2
    
    h = 1./N
    
    A = D2
    b = np.power(h,2.)*f_RHS(x[1:N])
    b[0] -= y[0]
    b[-1] -= y[N]
    
    y[1:-1] = np.linalg.solve(A,b)
    
    plt.figure(figsize=(L,L))
    
    plt.plot(x,y,'r.',ms=12,alpha=0.5,label=r'$y_i$')
    
    if f_RHS_id==0:
        y_exact = lambda t: np.power(t,3)/6.+(17.*t)/6-1.
        tt = np.linspace(0,1,100)
        plt.plot(tt, y_exact(tt),'b-',label=r'$y(t)$')
    
    plt.grid()
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()
    
interact(plot_numerical_solution_finite_difference,f_input=radio_button_function,N=(3,100,1), L=(3,10,1))


interactive(children=(RadioButtons(description='Function:', options=(('x', (0, <function <lambda> at 0x1366c1e…

<function __main__.plot_numerical_solution_finite_difference(f_input, N=5, L=5)>

<div id='acknowledgements' />

# Acknowledgements
[Back to TOC](#toc)

- _Material originally created by professor  Claudio Torres_ (`ctorres@inf.utfsm.cl`) _ DI UTFSM. June 2024.
- _Update June 2024 - v1.01 - C.Torres_ : Adding more flexibility for visualization of numerical solutions.